## FINAL CODE FOR CONTINUOUS CONTRACTS

In [59]:
## TO GET THE HOLIDAYS LIST

import numpy as np
from datetime import date, timedelta
import datetime as dt

start = date(2021, 10, 1)
end = date(2023, 10, 12)

from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, \
    USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, \
    USLaborDay, USThanksgivingDay


class USTradingCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
        USMartinLutherKingJr,
        USPresidentsDay,
        GoodFriday,
        USMemorialDay,
        Holiday('USIndependenceDay', month=7, day=4, observance=nearest_workday),
        USLaborDay,
        USThanksgivingDay,
        Holiday('Christmas', month=12, day=25, observance=nearest_workday)
    ]


def get_trading_close_holidays(year):
    inst = USTradingCalendar()

    return inst.holidays(dt.datetime(year-1, 12, 31), dt.datetime(year, 12, 31))


if __name__ == '__main__':
    holiday_list = []
    year = 2021
    for i in range(3):
        holiday_list.append(get_trading_close_holidays(year).date)
        year+=1

new_list = []
for i in range(len(holiday_list[:])):
    for j in range(len(holiday_list[i][:])):
        new_list.append(holiday_list[i][j])
new_list = [date_obj.strftime('%Y%m%d') for date_obj in new_list]
holiday_set = sorted(list(set(new_list)))
print(holiday_set)
# print(np.busday_count(start, end, holidays=holiday_set))

['20210101', '20210118', '20210215', '20210402', '20210531', '20210705', '20210906', '20211125', '20211224', '20211231', '20220117', '20220221', '20220415', '20220530', '20220704', '20220905', '20221124', '20221226', '20230102', '20230116', '20230220', '20230407', '20230529', '20230704', '20230904', '20231123', '20231225']


In [61]:
import pandas as pd
import warnings
import numpy as np
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")

# Enter file path containing raw data
options_data = pd.read_csv(r"C:\Data\USData\SPY-Opt-2023-09-12_1min.csv")

# Sort the options data by date and time
options_data['Date'] = pd.to_datetime(options_data['Date'])
options_data['Expiry_Dates'] = pd.to_datetime(options_data['Ticker'].str[5:11],format='%y%m%d')
options_data['Strike'] = options_data['Ticker'].str[-8:-3]
# options_data = options_data.sort_values(by=['Date', 'Expiry_Dates', 'Strike'])

# Create a list of unique expiration dates
expiration_dates = options_data['Expiry_Dates'].unique()

# Initialize a DataFrame to store the daily continuous contracts
daily_continuous_contracts = []

# Iterate through each trading day
for trading_day in tqdm(pd.date_range(start=options_data['Date'].min(), end=options_data['Date'].max(), freq='B')):
    # Filter options data for the current trading day
    daily_data = options_data[(options_data['Date'].dt.date == trading_day.date())]

    for expiration_date in expiration_dates:
        contract_data = daily_data[daily_data['Expiry_Dates'] == expiration_date]
        start,end = contract_data['Date'].dt.date.unique(),contract_data['Expiry_Dates'].dt.date.unique()
        if contract_data.empty == True:
            continue
        start,end = contract_data['Date'].dt.date.unique(),contract_data['Expiry_Dates'].dt.date.unique()
        start = [date_obj.strftime('%Y-%m-%d') for date_obj in start]
        end = [date_obj.strftime('%Y-%m-%d') for date_obj in end]
        dte = np.busday_count(start,end,holidays=holiday_set)
        # print(expiration_date,start,end,dte)
        contract_data['DTE'] = dte[0]
        
        daily_continuous_contracts.append(contract_data)

# Concatenate all daily continuous contracts into a single DataFrame
daily_continuous_contract = pd.concat(daily_continuous_contracts)

# Sort the continuous contract by date and time
daily_continuous_contract = daily_continuous_contract.sort_values(by=['Date', 'Expiry_Dates', 'Strike'])

# Save the daily continuous contract to a CSV file
# daily_continuous_contract.to_csv(r"C:\Data\USData\Continuous_Contracts\AllContracts\\Daily_continuous_SPY_options_data.csv", index=False)

print("Daily continuous contracts created and saved.")


  0%|          | 0/197 [00:00<?, ?it/s]

Daily continuous contracts created and saved.


In [63]:
weekly_groups = daily_continuous_contract.groupby('DTE')
unique_values = list(daily_continuous_contract['DTE'].unique())
unique_values = sorted([a for a in unique_values if a>=0 and a<=5])
for i in sorted(unique_values):
    print(f'DTE {i}')
    dte_df = weekly_groups.get_group(i).sort_values(by=['Date']).reset_index(drop=True)
    dte_df['New_Ticker'] = dte_df['Ticker'].str[:5] + f'D{i}' + dte_df['Ticker'].str[11:]
    display(dte_df)
    # dte_df.to_csv(rf"C:\Data\USData\Continuous_Contracts\DailyContracts\DTE{i}.csv",index=False)

DTE 0


,Ticker,Date,Time,open,high,low,close,volume,vwap,transactions,otc,Expiry_Dates,Strike,DTE,New_Ticker
0,O:SPY230912C00360000,2023-09-12,13:30:00,87.46,87.46,87.46,87.46,1,87.4600,1,NaN,2023-09-12,00360,0,O:SPYD0C00360000
1,O:SPY230912P00448000,2023-09-12,19:19:00,2.40,2.41,2.36,2.39,32,2.3991,8,NaN,2023-09-12,00448,0,O:SPYD0P00448000
2,O:SPY230912P00448000,2023-09-12,19:18:00,2.32,2.40,2.32,2.34,60,2.3645,8,NaN,2023-09-12,00448,0,O:SPYD0P00448000
3,O:SPY230912P00448000,2023-09-12,19:17:00,2.30,2.38,2.29,2.29,13,2.3600,3,NaN,2023-09-12,00448,0,O:SPYD0P00448000
4,O:SPY230912P00448000,2023-09-12,19:16:00,2.37,2.37,2.26,2.30,36,2.2894,9,NaN,2023-09-12,00448,0,O:SPYD0P00448000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237463,O:SPY231011C00432000,2023-10-11,13:58:00,3.70,3.72,3.70,3.72,60,3.7133,5,NaN,2023-10-11,00432,0,O:SPYD0C00432000
237464,O:SPY231011C00432000,2023-10-11,13:59:00,3.80,3.80,3.80,3.80,1,3.8000,1,NaN,2023-10-11,00432,0,O:SPYD0C00432000
237465,O:SPY231011C00432000,2023-10-11,14:00:00,3.72,3.72,3.44,3.56,5,3.5680,3,NaN,2023-10-11,00432,0,O:SPYD0C00432000
237466,O:SPY231011C00432000,2023-10-11,13:39:00,4.15,4.34,4.15,4.34,32,4.1569,3,NaN,2023-10-11,00432,0,O:SPYD0C00432000


DTE 1


,Ticker,Date,Time,open,high,low,close,volume,vwap,transactions,otc,Expiry_Dates,Strike,DTE,New_Ticker
0,O:SPY230912C00360000,2023-09-11,16:25:00,87.34,87.34,87.34,87.34,1,87.3400,1,NaN,2023-09-12,00360,1,O:SPYD1C00360000
1,O:SPY230912C00449000,2023-09-11,18:17:00,0.77,0.80,0.76,0.78,196,0.7762,28,NaN,2023-09-12,00449,1,O:SPYD1C00449000
2,O:SPY230912C00449000,2023-09-11,18:16:00,0.83,0.83,0.78,0.78,178,0.8038,41,NaN,2023-09-12,00449,1,O:SPYD1C00449000
3,O:SPY230912C00449000,2023-09-11,18:15:00,0.83,0.85,0.81,0.82,186,0.8369,43,NaN,2023-09-12,00449,1,O:SPYD1C00449000
4,O:SPY230912C00449000,2023-09-11,18:14:00,0.83,0.84,0.81,0.82,425,0.8286,32,NaN,2023-09-12,00449,1,O:SPYD1C00449000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276147,O:SPY231011P00430000,2023-10-10,15:58:00,0.20,0.21,0.20,0.21,67,0.2069,18,NaN,2023-10-11,00430,1,O:SPYD1P00430000
276148,O:SPY231011P00430000,2023-10-10,15:59:00,0.21,0.21,0.21,0.21,23,0.2100,10,NaN,2023-10-11,00430,1,O:SPYD1P00430000
276149,O:SPY231011P00430000,2023-10-10,16:00:00,0.21,0.21,0.20,0.20,8,0.2088,4,NaN,2023-10-11,00430,1,O:SPYD1P00430000
276150,O:SPY231011P00430000,2023-10-10,15:50:00,0.23,0.23,0.22,0.22,21,0.2210,3,NaN,2023-10-11,00430,1,O:SPYD1P00430000


DTE 2


...

,Ticker,Date,Time,open,high,low,close,volume,vwap,transactions,otc,Expiry_Dates,Strike,DTE,New_Ticker
0,O:SPY230912C00370000,2023-09-07,15:01:00,74.55,74.55,74.55,74.55,10,74.5500,1,NaN,2023-09-12,00370,3,O:SPYD3C00370000
1,O:SPY230912P00446000,2023-09-07,13:38:00,3.45,3.63,3.45,3.58,122,3.5070,8,NaN,2023-09-12,00446,3,O:SPYD3P00446000
2,O:SPY230912P00446000,2023-09-07,13:37:00,3.47,3.48,3.38,3.48,95,3.4335,16,NaN,2023-09-12,00446,3,O:SPYD3P00446000
3,O:SPY230912P00446000,2023-09-07,13:36:00,3.44,3.52,3.44,3.51,29,3.5072,6,NaN,2023-09-12,00446,3,O:SPYD3P00446000
4,O:SPY230912P00446000,2023-09-07,13:35:00,3.56,3.56,3.46,3.49,61,3.5297,14,NaN,2023-09-12,00446,3,O:SPYD3P00446000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181494,O:SPY231011C00423000,2023-10-06,15:05:00,4.04,4.04,4.01,4.01,40,4.0175,2,NaN,2023-10-11,00423,3,O:SPYD3C00423000
181495,O:SPY231011C00423000,2023-10-06,15:06:00,4.07,4.07,4.07,4.07,40,4.0700,1,NaN,2023-10-11,00423,3,O:SPYD3C00423000
181496,O:SPY231011C00423000,2023-10-06,15:07:00,4.14,4.14,4.14,4.14,75,4.1400,1,NaN,2023-10-11,00423,3,O:SPYD3C00423000
181497,O:SPY231011C00423000,2023-10-06,15:00:00,3.77,3.81,3.77,3.81,5,3.7860,2,NaN,2023-10-11,00423,3,O:SPYD3C00423000


DTE 4


,Ticker,Date,Time,open,high,low,close,volume,vwap,transactions,otc,Expiry_Dates,Strike,DTE,New_Ticker
0,O:SPY230912P00379000,2023-09-06,17:33:00,0.02,0.02,0.02,0.02,1,0.0200,1,NaN,2023-09-12,00379,4,O:SPYD4P00379000
1,O:SPY230912C00449000,2023-09-06,14:19:00,1.64,1.65,1.64,1.65,45,1.6438,3,NaN,2023-09-12,00449,4,O:SPYD4C00449000
2,O:SPY230912C00449000,2023-09-06,14:18:00,1.71,1.71,1.71,1.71,1,1.7100,1,NaN,2023-09-12,00449,4,O:SPYD4C00449000
3,O:SPY230912C00449000,2023-09-06,14:16:00,1.57,1.67,1.57,1.67,60,1.6637,3,NaN,2023-09-12,00449,4,O:SPYD4C00449000
4,O:SPY230912C00449000,2023-09-06,14:15:00,1.54,1.54,1.53,1.53,2,1.5350,2,NaN,2023-09-12,00449,4,O:SPYD4C00449000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147129,O:SPY231011P00421000,2023-10-05,15:31:00,3.41,3.41,3.41,3.41,2,3.4100,2,NaN,2023-10-11,00421,4,O:SPYD4P00421000
147130,O:SPY231011P00421000,2023-10-05,15:30:00,3.30,3.31,3.30,3.31,2,3.3050,2,NaN,2023-10-11,00421,4,O:SPYD4P00421000
147131,O:SPY231011P00421000,2023-10-05,15:27:00,3.18,3.19,3.18,3.19,11,3.1809,3,NaN,2023-10-11,00421,4,O:SPYD4P00421000
147132,O:SPY231011P00421000,2023-10-05,15:36:00,3.17,3.17,3.17,3.17,80,3.1700,1,NaN,2023-10-11,00421,4,O:SPYD4P00421000


DTE 5


,Ticker,Date,Time,open,high,low,close,volume,vwap,transactions,otc,Expiry_Dates,Strike,DTE,New_Ticker
0,O:SPY230912P00379000,2023-09-05,13:31:00,0.02,0.02,0.02,0.02,205,0.0200,4,NaN,2023-09-12,00379,5,O:SPYD5P00379000
1,O:SPY230912P00451000,2023-09-05,16:28:00,2.64,2.64,2.64,2.64,1,2.6400,1,NaN,2023-09-12,00451,5,O:SPYD5P00451000
2,O:SPY230912P00451000,2023-09-05,16:29:00,2.64,2.64,2.63,2.63,3,2.6367,2,NaN,2023-09-12,00451,5,O:SPYD5P00451000
3,O:SPY230912P00451000,2023-09-05,16:37:00,2.50,2.50,2.50,2.50,4,2.5000,1,NaN,2023-09-12,00451,5,O:SPYD5P00451000
4,O:SPY230912P00451000,2023-09-05,16:38:00,2.51,2.51,2.51,2.51,30,2.5100,10,NaN,2023-09-12,00451,5,O:SPYD5P00451000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144019,O:SPY231011P00420000,2023-10-04,19:34:00,1.92,1.92,1.90,1.90,16,1.9187,2,NaN,2023-10-11,00420,5,O:SPYD5P00420000
144020,O:SPY231011P00420000,2023-10-04,19:33:00,1.93,1.93,1.93,1.93,2,1.9300,1,NaN,2023-10-11,00420,5,O:SPYD5P00420000
144021,O:SPY231011P00420000,2023-10-04,19:32:00,1.93,1.93,1.90,1.90,37,1.9049,4,NaN,2023-10-11,00420,5,O:SPYD5P00420000
144022,O:SPY231011P00420000,2023-10-04,19:41:00,1.75,1.78,1.75,1.78,7,1.7586,2,NaN,2023-10-11,00420,5,O:SPYD5P00420000


## TO GET WORKING DAYS BETWEEN TWO DATES AND CONSIDERING HOLIDAYS

In [46]:
import numpy as np
from datetime import date, timedelta
start = date(2023, 10, 1)
end = date(2023, 10, 12)

import datetime as dt

from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, \
    USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, \
    USLaborDay, USThanksgivingDay


class USTradingCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
        USMartinLutherKingJr,
        USPresidentsDay,
        GoodFriday,
        USMemorialDay,
        Holiday('USIndependenceDay', month=7, day=4, observance=nearest_workday),
        USLaborDay,
        USThanksgivingDay,
        Holiday('Christmas', month=12, day=25, observance=nearest_workday)
    ]


def get_trading_close_holidays(year):
    inst = USTradingCalendar()

    return inst.holidays(dt.datetime(year-1, 12, 31), dt.datetime(year, 12, 31))


if __name__ == '__main__':
    holiday_list = []
    holiday_list.append(get_trading_close_holidays(2023).date)

new_list = []
for i in range(len(holiday_list[0])):
    new_list.append(holiday_list[0][i])
new_list = [date_obj.strftime('%Y%m%d') for date_obj in new_list]
holiday_set = new_list

print(np.busday_count(start, end, holidays=holiday_set))

8


## ROUGH CODE TO CREATE CONTINUOUS CONTRACTS

In [64]:
# import pandas as pd
# import warnings
# warnings.filterwarnings("ignore")
# # Sample options data for demonstration (Replace with your actual data source)
# options_data = pd.read_csv(r"C:\Data\USData\SPY-Opt-2023-10-01_1min.csv")

# # Sort the options data by date and time
# options_data['Date'] = pd.to_datetime(options_data['Date'])
# options_data['Expiry_Dates'] = pd.to_datetime(options_data['Ticker'].str[5:11],format='%y%m%d')
# options_data['Strike'] = options_data['Ticker'].str[-8:-3]
# # options_data = options_data.sort_values(by=['Date', 'Expiry_Dates', 'Strike'])

# # Create a list of unique expiration dates
# expiration_dates = options_data['Expiry_Dates'].unique()

# # Initialize a DataFrame to store the daily continuous contracts
# daily_continuous_contracts = []

# # Iterate through each trading day
# for trading_day in pd.date_range(start=options_data['Date'].min(), end=options_data['Date'].max(), freq='B'):
#     # Filter options data for the current trading day
#     daily_data = options_data[(options_data['Date'].dt.date == trading_day.date())]
#     start,end = contract_data['Date'].dt.date.unique(),contract_data['Expiry_Dates'].dt.date.unique()
#     # Handle rolling over to the next contract (replace this logic with your specific rules)
#     # For simplicity, let's assume we roll over to the next expiration date
#     for expiration_date in expiration_dates:
#         contract_data = daily_data[daily_data['Expiry_Dates'] == expiration_date]
#         display(contract_data)
#         if contract_data.empty == True:
#             continue
#         # Calculate the next expiration date
#         next_expiration_date = expiration_dates[expiration_dates > expiration_date].min()
#         contract_data['Next_Expiry_Dates'] = next_expiration_date
#         start,end = contract_data['Date'].dt.date.unique(),contract_data['Expiry_Dates'].dt.date.unique()
#         start = [date_obj.strftime('%Y-%m-%d') for date_obj in start]
#         end = [date_obj.strftime('%Y-%m-%d') for date_obj in end]
#         dte = np.busday_count(start,end,holidays=holiday_set)
#         print(expiration_date,start,end,dte)
#         contract_data['DTE'] = dte[0]
        
#         # Add the rolled-over contract to the daily continuous contracts
#         daily_continuous_contracts.append(contract_data)

# # Concatenate all daily continuous contracts into a single DataFrame
# daily_continuous_contract = pd.concat(daily_continuous_contracts)

# # Sort the continuous contract by date and time
# daily_continuous_contract = daily_continuous_contract.sort_values(by=['Date', 'Expiry_Dates', 'Strike'])

# # Save the daily continuous contract to a CSV file
# daily_continuous_contract.to_csv(r"C:\\users\\admin\\desktop\\daily_continuous_SPY_options_data.csv", index=False)

# print("Daily continuous contracts created and saved.")
